In [1]:
# Imports
import tensorflow as tf
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd
import matplotlib.pyplot as plt
import inspect
from tqdm import tqdm

# Set batch size for training and validation
batch_size = 32

In [2]:
# List all available models
model_dictionary = {m[0]:m[1] for m in inspect.getmembers(tf.keras.applications, inspect.isfunction)}
print(model_dictionary)

{'DenseNet121': <function DenseNet121 at 0x7feed26fa9e0>, 'DenseNet169': <function DenseNet169 at 0x7feed26faa70>, 'DenseNet201': <function DenseNet201 at 0x7feed26fab00>, 'EfficientNetB0': <function EfficientNetB0 at 0x7feed2700710>, 'EfficientNetB1': <function EfficientNetB1 at 0x7feed27007a0>, 'EfficientNetB2': <function EfficientNetB2 at 0x7feed2700830>, 'EfficientNetB3': <function EfficientNetB3 at 0x7feed27008c0>, 'EfficientNetB4': <function EfficientNetB4 at 0x7feed2700950>, 'EfficientNetB5': <function EfficientNetB5 at 0x7feed27009e0>, 'EfficientNetB6': <function EfficientNetB6 at 0x7feed2700a70>, 'EfficientNetB7': <function EfficientNetB7 at 0x7feed2700b00>, 'InceptionResNetV2': <function InceptionResNetV2 at 0x7feed27005f0>, 'InceptionV3': <function InceptionV3 at 0x7feed270b200>, 'MobileNet': <function MobileNet at 0x7feed270b7a0>, 'MobileNetV2': <function MobileNetV2 at 0x7feed270bdd0>, 'MobileNetV3Large': <function MobileNetV3Large at 0x7feed271a170>, 'MobileNetV3Small': <

In [3]:
# Download the training and validation data
# Set up training and validation dataset
train_ds_224 = tf.keras.preprocessing.image_dataset_from_directory(
    "/Users/brian/ml/DetectSelfie/data/Selfie-Image-Detection-Dataset/Training_data_reduced",
    labels="inferred",
    label_mode="int",
    color_mode="rgb",
    seed=1337,
    image_size=(224, 224),
    batch_size=batch_size,
)
val_ds_224 = tf.keras.preprocessing.image_dataset_from_directory(
    "/Users/brian/ml/DetectSelfie/data/Selfie-Image-Detection-Dataset-Grayscale/Validation_data",
    labels="inferred",
    label_mode="int",
    color_mode="rgb",
    seed=1337,
    image_size=(224, 224),
    batch_size=batch_size,
)

Found 20000 files belonging to 2 classes.
Found 7862 files belonging to 2 classes.


In [4]:
# Set up training and validation dataset
train_ds_331 = tf.keras.preprocessing.image_dataset_from_directory(
    "/Users/brian/ml/DetectSelfie/data/Selfie-Image-Detection-Dataset/Training_data_reduced",
    labels="inferred",
    label_mode="int",
    color_mode="rgb",
    seed=1337,
    image_size=(331, 331),
    batch_size=batch_size,
)
val_ds_331 = tf.keras.preprocessing.image_dataset_from_directory(
    "/Users/brian/ml/DetectSelfie/data/Selfie-Image-Detection-Dataset-Grayscale/Validation_data",
    labels="inferred",
    label_mode="int",
    color_mode="rgb",
    seed=1337,
    image_size=(331, 331),
    batch_size=batch_size,
)

Found 20000 files belonging to 2 classes.
Found 7862 files belonging to 2 classes.


In [5]:
# Number of training examples and labels
num_train = int(train_ds_224.__len__()) * 32
num_validation = int(val_ds_224.__len__()) * 32
num_classes = len(train_ds_224.class_names)
num_iterations = train_ds_224.__len__()

# Print important info
print(f'Num train images: {num_train} \
        \nNum validation images: {num_validation} \
        \nNum classes: {num_classes} \
        \nNum iterations per epoch: {num_iterations}')

Num train images: 20000         
Num validation images: 7872         
Num classes: 2         
Num iterations per epoch: 625


In [6]:
# Set up data augmentation preprocessing layers
data_augmentation = keras.Sequential(
    [
        layers.experimental.preprocessing.RandomFlip("horizontal"),
        layers.experimental.preprocessing.RandomRotation(0.1),
        layers.experimental.preprocessing.RandomZoom(0.3, 0.3),
        layers.experimental.preprocessing.RandomTranslation(0.2, 0.2),
    ]
)

In [7]:
# Loop over each model available in Keras
model_benchmarks = {'model_name': [], 'num_model_params': [], 'validation_accuracy': []}

for model_name, model in tqdm(model_dictionary.items()):
    # Special handling for "NASNetLarge" since it requires input images with size (331,331)
    if 'NASNetLarge' in model_name:
        input_shape=(331,331,3)
        train_processed = train_ds_331
        validation_processed = val_ds_331
        inputs = keras.Input(shape=(331, 331, 3))
    else:
        input_shape=(224,224,3)
        train_processed = train_ds_224
        validation_processed = val_ds_224
        inputs = keras.Input(shape=(224, 224, 3))
        
    # load the pre-trained model with global average pooling as the last layer and freeze the model weights
    pre_trained_model = model(include_top=False, input_shape=input_shape)
    pre_trained_model.trainable = False

    # Create new model on top
    x = data_augmentation(inputs)  # Apply random data augmentation
    
    # Pre-trained Xception weights requires that input be normalized
    # from (0, 255) to a range (-1., +1.), the normalization layer
    # does the following, outputs = (inputs - mean) / sqrt(var)

    norm_layer = keras.layers.experimental.preprocessing.Normalization()
    mean = np.array([127.5] * 3)
    var = mean ** 2

    # Scale inputs to [-1, +1]
    x = norm_layer(x)
    norm_layer.set_weights([mean, var])

    # We make sure that the base_model is running in inference mode here,
    # by passing `training=False`.
    x = pre_trained_model(x, training=False)
    
    # Convert features of shape `base_model.output_shape[1:]` to vectors
    x = keras.layers.GlobalAveragePooling2D()(x)

    # A Dense classifier with a single unit (binary classification)
    x = keras.layers.Dropout(0.2)(x)  # Regularize with dropout
    outputs = keras.layers.Dense(1, activation="sigmoid")(x)
    clf_model = keras.Model(inputs, outputs)
    
    # Train model
    epochs = 3

    clf_model.summary()
    clf_model.compile(optimizer=keras.optimizers.Adam(),
                  loss=keras.losses.BinaryCrossentropy(from_logits=True),
                  metrics=[keras.metrics.BinaryAccuracy()])
    train_history = clf_model.fit(train_processed, epochs=epochs, validation_data=validation_processed)

  0%|          | 0/28 [00:00<?, ?it/s]

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
sequential (Sequential)      (None, 224, 224, 3)       0         
_________________________________________________________________
normalization (Normalization (None, 224, 224, 3)       7         
_________________________________________________________________
densenet121 (Functional)     (None, 7, 7, 1024)        7037504   
_________________________________________________________________
global_average_pooling2d (Gl (None, 1024)              0         
_________________________________________________________________
dropout (Dropout)            (None, 1024)              0         
_________________________________________________________________
dense (Dense)                (None, 1)                 1025  

  4%|▎         | 1/28 [2:17:15<61:46:06, 8235.79s/it]

51879936/51877672 [==============================] - 2s 0us/step
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
sequential (Sequential)      (None, 224, 224, 3)       0         
_________________________________________________________________
normalization_1 (Normalizati (None, 224, 224, 3)       7         
_________________________________________________________________
densenet169 (Functional)     (None, 7, 7, 1664)        12642880  
_________________________________________________________________
global_average_pooling2d_1 ( (None, 1664)              0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 1664)              0         
____________________________________________________________

  7%|▋         | 2/28 [4:39:10<60:40:02, 8400.09s/it]

...

6701056/6698480 [==============================] - 0s 0us/step
Model: "model_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_33 (InputLayer)        [(None, 224, 224, 3)]     0         
_________________________________________________________________
sequential (Sequential)      (None, 224, 224, 3)       0         
_________________________________________________________________
normalization_24 (Normalizat (None, 224, 224, 3)       7         
_________________________________________________________________
MobilenetV3small (Functional (None, 7, 7, 1024)        1529968   
_________________________________________________________________
global_average_pooling2d_16  (None, 1024)              0         
_________________________________________________________________
dropout_16 (Dropout)         (None, 1024)              0         
_____________________________________________________________

 61%|██████    | 17/28 [37:43:45<13:12:41, 4323.78s/it]

293617664/343610240 [========================>.....] - ETA: 2s

 61%|██████    | 17/28 [37:44:15<24:25:06, 7991.47s/it]


OSError: [Errno 28] No space left on device

In [ ]:
# Calculate all relevant metrics
model_benchmarks['model_name'].append(model_name)
model_benchmarks['num_model_params'].append(pre_trained_model.count_params())
model_benchmarks['validation_accuracy'].append(history.history['val_accuracy'][-1])m bn

In [ ]:
# Convert Results to DataFrame for easy viewing
benchmark_df = pd.DataFrame(model_benchmarks)

# sort in ascending order of num_model_params column
benchmark_df.sort_values('num_model_params', inplace=True)

# write results to csv file
benchmark_df.to_csv('benchmark_df.csv', index=False)
benchmark_df

In [ ]:
# Loop over each row and plot the num_model_params vs validation_accuracy
markers=[".",",","o","v","^","<",">","1","2","3","4","8","s","p","P","*","h","H","+","x","X","D","d","|","_",4,5,6,7,8,9,10,11]
plt.figure(figsize=(7,5))

for row in benchmark_df.itertuples():
    plt.scatter(row.num_model_params, row.validation_accuracy, label=row.model_name, marker=markers[row.Index], s=150, linewidths=2)
    
plt.xscale('log')
plt.xlabel('Number of Parameters in Model')
plt.ylabel('Validation Accuracy after 3 Epochs')
plt.title('Accuracy vs Model Size')

# Move legend out of the plot
plt.legend(bbox_to_anchor=(1, 1), loc='upper left');